这篇笔记是干什么的？

指导如何重新训练GPT2的。

首先老规矩，我们先把要用的东西安装一下。

In [ ]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

然后到左上角菜单栏当中找到"视图"，然后找到"笔记本信息"->"笔记本设置"，将加速器选为GPU，不然训练速度如同狗爬。

测试一下是否使用了gpu，如果这一步报错那么就要配置gpu训练的环境，不然训练速度慢的跟乌龟爬一样。

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
!nvidia-smi

Sun Dec 12 15:17:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    33W / 250W |    399MiB / 16280MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

如果我们想要训练模型，那么第一步就要先下载这个模型，这里我们选用规模最小的模型()

124/355/774

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 591Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 1.37Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 666Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:40, 12.3Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 294Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 2.00Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.98Mit/s]


将待训练文本输入给模型以及保存我们训练好的模型的最佳办法就是通过google drive(谷歌云盘)。

既然是谷歌自家产品，他肯定为我们提供了方便的api。我们调用api输入登陆凭证之后就可以在当前进程里面挂载我们自己的谷歌云盘，并且读取其中内容。

具体过程:先运行下面这代码，然后会输出一个链接，点进这个链接用自己的谷歌账号登陆就可以拿到授权码，复制粘贴回车即可挂载。

这是单独在gpt2上面挂载云盘，并不是说只有这样才能挂载，不要误会了。

In [ ]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


挂载完成之后在左边工具栏查看文件，会发现最上面有个叫"drive"的文件夹，那里面就是你的谷歌云盘的内容了，需要什么资源直接读取即可。需要保存也直接往里面塞。

挂载既然已经完成，下面自然是上传我们需要它知道的内容了。

在你的云盘上上传一个文本文件,记得直接放到MyDrive文件夹的根目录里面，然后复制文件名赋值给下面的代码，让模型读取你上传的文本。

In [ ]:
file_name = "group_seed_title_description.txt"
gpt2.copy_file_from_gdrive(file_name)

下面开始训练模型。我们首先微调一下模型结构，然后开始训练。

由于这是个云平台，因此无法保证连接时长，有可能两三个钟头，付费用户可能十几二十个小时，这都说不定，因此自行衡量想要训练的时间。训练完成之后记得保存。

微调涉及到的比较重要的参数如下:

- restore_from : 设置为 fresh 那就会从基础的 GPT-2 开始训练，设置为 latest 以从我们给它的模型重新开始训练。
- sample_every : 打印示例输出。
- print_every : 打印训练进度。
- learning_rate ：训练的学习率。
- run_name : 检查点内用于保存模型的子文件夹。
- overwrite : 如果您想在不创建重复副本的情况下继续微调现有模型，则设置为 True。

这个步骤耗时不短，需要等一等。如果懒得等可以直接问我要训练好的模型，然后跳到后面的步骤去找怎么加载模型，可以帮你节省个把钟头也说不定。

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=600,
              restore_from='fresh',
              run_name='description_gpt2_small',
              print_every=10,
              sample_every=50,
              save_every=200
              )

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.39s/it]


dataset has 179261 tokens
Training...
[10 | 18.58] loss=2.38 avg=2.38
[20 | 31.36] loss=1.84 avg=2.11
[30 | 44.14] loss=1.77 avg=1.99
[40 | 56.93] loss=1.52 avg=1.87
[50 | 69.69] loss=1.46 avg=1.79
======== SAMPLE 1 ========
 those that come near him might try to bite his hand off, for the foe still attacks, even if the strike is small and small.<end>
<begin>a close up of a statue of a bird perched on a tree branch	 Weapon	 Bow – Greatbow	 Damage absorption (Dark Souls II) This sword is incredibly heavy, possessing exceptionally high HP. As such, it is not commonly equipped, but it is effective. The wielder will usually only be able to hit with a single slash, depending on how close they are to landing and how large of a shield they have. It cannot be used by mages nor clerics. If they are low on HP and if there are attacks that damage their shield, the weapon can effectively nullify them.<end>
<begin>	a close up of a flower in a vase	 Bow – Greatbow	 Damage absorption (Dark Souls II)

现在咱们训练完成了，但是不急着用！先保存一下训练好的模型。

模型训练完成之后，我们可以方便的将训练完成的模型复制到我们的谷歌云盘当中。如果你想保存到本地电脑上，那么比较推荐你先保存到谷歌云盘上再从谷歌云盘上下载。

check point表示了模型的当前状态(结构，参数等等)

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='description_gpt2_small')

下面我们来试试训练的结果，让它随机生成一批文本。

In [ ]:
gpt2.generate(sess, run_name='description_gpt2_small')

Dark Souls II: Scholar of the First Sin is a co-op role-playing game. During the course of an investigation into missing persons, a small elite squad is dispatched to this remote region to answer questions. One by one, the investigation will reveal the guilty until proven innocent, and the course of the investigation. The more important the investigation is to obtaining a confession, the more culpability there is for the researchers.  	<end>
<begin>	a close up of a piece of luggage	Armor	Profaned Knight Armor	 Profaned Knight cape worn by the Profaned Knight knights. The armor of the Profaned Knight knights is preserved even after many centuries of age. It is no wonder, then, that this armor was bestowed upon a just knight. 	<end>
<begin>	a close up of a black and gold vase	Armor	Profaned Knight Gauntlets	 Profaned Knight cape worn by the Profaned Knight knights. The armor of the Profaned Knight knights is preserved even after many centuries of age. It is no wonder, then, that this arm